In [1]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm 

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2024-03-07 11:51:27.664912: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 11:51:27.831143: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 11:51:27.899805: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBL

In [5]:
df = pd.read_csv('./tsla_merged_final.csv')
df

,Date,Content
0,2020-12-31,Tesla short sellers lost billions more on the ...
1,2020-12-31,Enphase joins Tesla as the latest clean tech f...
2,2020-12-29,"In 2021, the breadth of the competitive attack..."
3,2020-12-29,"In 2021, the breadth of the competitive attack..."
4,2020-12-28,"In 2021, the breadth of the competitive attack..."
...,...,...
12907,2015-02-13,Apple has assigned 100s of staffers to an elec...
12908,2015-01-28,"Apple, Microsoft and Tesla shops are helping h..."
12909,2015-01-17,Elon Musk says Tesla won't be profitable until...
12910,2015-01-14,Elon Musk says Tesla won't be profitable until...


In [6]:
new_df = df.drop_duplicates() # 刪除重複的資料
new_df['label'] = ''
new_df['score'] = ''
new_df

/tmp/ipykernel_888293/22623641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['label'] = ''
/tmp/ipykernel_888293/22623641.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['score'] = ''


,Date,Content,label,score
0,2020-12-31,Tesla short sellers lost billions more on the ...,,
1,2020-12-31,Enphase joins Tesla as the latest clean tech f...,,
2,2020-12-29,"In 2021, the breadth of the competitive attack...",,
4,2020-12-28,"In 2021, the breadth of the competitive attack...",,
5,2020-12-25,Elon Musk says it’s “impossible” to take Tesla...,,
...,...,...,...,...
12907,2015-02-13,Apple has assigned 100s of staffers to an elec...,,
12908,2015-01-28,"Apple, Microsoft and Tesla shops are helping h...",,
12909,2015-01-17,Elon Musk says Tesla won't be profitable until...,,
12910,2015-01-14,Elon Musk says Tesla won't be profitable until...,,


In [7]:
new_df.to_csv('tsla_merged_res_2.csv', index=False)

In [8]:
new_df = pd.read_csv('./tsla_merged_res_2.csv')
new_df

,Date,Content,label,score
0,2020-12-31,Tesla short sellers lost billions more on the ...,NaN,NaN
1,2020-12-31,Enphase joins Tesla as the latest clean tech f...,NaN,NaN
2,2020-12-29,"In 2021, the breadth of the competitive attack...",NaN,NaN
3,2020-12-28,"In 2021, the breadth of the competitive attack...",NaN,NaN
4,2020-12-25,Elon Musk says it’s “impossible” to take Tesla...,NaN,NaN
...,...,...,...,...
11144,2015-02-13,Apple has assigned 100s of staffers to an elec...,NaN,NaN
11145,2015-01-28,"Apple, Microsoft and Tesla shops are helping h...",NaN,NaN
11146,2015-01-17,Elon Musk says Tesla won't be profitable until...,NaN,NaN
11147,2015-01-14,Elon Musk says Tesla won't be profitable until...,NaN,NaN


In [9]:
# 導入pretrianed model
deberta = pipeline("text-classification", model="mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/568M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [10]:
i=0
for line in tqdm(new_df['Content'], total=len(new_df)):
    if isinstance(line, str):
        res = deberta(line)
        new_df.loc[i, 'label'] = res[0]['label']
        new_df.loc[i, 'score'] = res[0]['score']
    else:
        print(f"Ignoring invalid data: {line}")
    i += 1
    
print("predict done")

  0%|          | 0/11149 [00:00<?, ?it/s]/tmp/ipykernel_888293/1126264365.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'negative' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  new_df.loc[i, 'label'] = res[0]['label']
100%|██████████| 11149/11149 [04:11<00:00, 44.41it/s]

predict done


In [11]:
print(new_df.head(20))

          Date                                            Content     label  \
0   2020-12-31  Tesla short sellers lost billions more on the ...  negative   
1   2020-12-31  Enphase joins Tesla as the latest clean tech f...  positive   
2   2020-12-29  In 2021, the breadth of the competitive attack...  positive   
3   2020-12-28  In 2021, the breadth of the competitive attack...  positive   
4   2020-12-25  Elon Musk says it’s “impossible” to take Tesla...   neutral   
5   2020-12-25  Elon Musk has made millionaires out of Tesla's...   neutral   
6   2020-12-25  Elon Musk says he tried to approach Apple year...   neutral   
7   2020-12-23  Elon Musk says he tried to approach Apple year...   neutral   
8   2020-12-22  Elon Musk says he once tried to sell Tesla to ...   neutral   
9   2020-12-22  Elon Musk says he tried to approach Apple year...   neutral   
10  2020-12-22  Apple’s reported foray into making its own sel...   neutral   
11  2020-12-21  It didn’t take long for Tesla’s big 

In [12]:
new_df.to_csv('tsla_merged_res_2.csv', index=False)